In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

# sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path

In [ ]:
from dotenv import load_dotenv
import os

# dotenv_path = "../.secret_env"
# load_dotenv(dotenv_path=dotenv_path)
load_dotenv()

T_TOKEN = os.getenv("T_TOKEN")
print(T_TOKEN)

In [ ]:
TG_TOKEN = os.getenv("TG_TOKEN")
print(TG_TOKEN)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go


In [ ]:
from market_oracle_lib import hello
from market_oracle_lib.data import t_bank, yfinance, apimoex
from market_oracle_lib.data.data_funcs import create_data_loaders, create_data_frames, create_data_sets, RU_DEFAULT_SYMBOLS
from help_src.patterns.src import isPivot, calculate_point_pos, collect_channel, refine_channel, visualize_data, calculate_breakout_series, add_pivot_column
from help_src.patterns.channel_analysis import analyze_price_channels


# Ноутбук для дебага алгоритмического распознавания патернов

## Пример получения данных

In [ ]:
# train_dl, val_dl, test_dl = create_data_frames(t_bank.get_symbol_data, interval="1d",
#                                                token=T_TOKEN,
#                                             #    start_date="2024-01-01",
#                                             #    end_date="2024-09-02",
#                                                symbols=RU_DEFAULT_SYMBOLS)
# # train_dl
# # next(iter(train_dl))[0].shape, next(iter(train_dl))[1].shape

In [ ]:
# train_dl.head()

In [ ]:
one_sym_ser = t_bank.get_symbol_data(symbol=RU_DEFAULT_SYMBOLS[19], token=T_TOKEN,
                                     # start_date="2024-01-01",
                                     # end_date="2024-09-02",
                                     interval="1d").sort_values(by="Date", ascending=False).sort_values(by="Date", ascending=False)
one_sym_ser.head()

## Получение списка акций

In [ ]:
# YNDX, POLY
ticker = "T"
instrument_info = t_bank.find_instrument_by_ticker(ticker, T_TOKEN)
instrument_info

In [ ]:
from tinkoff.invest import Client

with Client(T_TOKEN) as client:
    tmp = t_bank._get_instrument_methods(client)[0]
    tmp2 = tmp[1]().instruments
    tmp3 = list(filter(lambda x: x.country_of_risk == "RU", tmp2))
    # print(tmp2)

In [ ]:
for share in tmp3:
    print(f"{share.ticker} - {share.name}")

## Работа с данными

In [ ]:
# plt.figure(figsize=(10, 5))
# plt.plot(one_sym_ser["Date"], one_sym_ser["Close"])
# plt.show()

In [ ]:
tmp_data = one_sym_ser[:1000].sort_values(by="Date", ascending=True)
# tmp_data
plt.figure(figsize=(10, 5))
plt.plot(tmp_data["Date"], tmp_data["Close"])
plt.show()

In [ ]:
tmp_data.head()

In [ ]:
window_size = 3 # Как в примере BreakOut_Indicator
# tmp_data.loc[:, 'isPivot'] = [isPivot(tmp_data, i, window_size) for i in range(len(tmp_data))]
# tmp_data.loc[:, 'pointpos'] = tmp_data.apply(lambda row: calculate_point_pos(row, low_col='Low', high_col='High', pivot_col='isPivot'), axis=1)

add_pivot_column(tmp_data, window_size)

# Отмечаем пивоты
pivot_highs = tmp_data[tmp_data['isPivot'] == 1]
pivot_lows = tmp_data[tmp_data['isPivot'] == 2]
# Также можно отметить пивоты типа 3, если они важны
pivot_both = tmp_data[tmp_data['isPivot'] == 3]


In [ ]:
visualize_data(tmp_data,
               pivot_highs=pivot_highs,
               pivot_lows=pivot_lows,
               pivot_both=pivot_both,
               candle=100,
               backcandles=40,
               window=3
)


In [ ]:
tmp_data[['Open', 'High', 'Low', 'Close', 'isPivot', 'pointpos']].head()
# tmp_data

### И эксремумы и один жкземпляр ценнового канала

In [ ]:
# Предположим, у вас есть df, candle, backcandles, window
df = tmp_data
candle = len(df) # Пример
backcandles = 200 # Пример
window = 3 # Пример

# Определяем диапазон для отображения: немного до начала окна анализа и немного после текущей свечи
# display_range_start = max(0, candle - backcandles - window - 10)
# display_range_end = min(len(df), candle + 20)
display_range_start, display_range_end = int(len(df) * (0/4)), len(df)
df_display = df.iloc[display_range_start:display_range_end]


# Вызываем вашу функцию
# sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h, st_idx, end_idx = collect_channel(df_display, candle, backcandles, window)
price_ch = collect_channel(df_display, candle, backcandles, window, should_refine=True)

# print(sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h, st_idx, end_idx)
print(price_ch)

In [ ]:
tmp_data
pivot_highs
pivot_lows
pivot_both
price_ch

In [ ]:
visualize_data(tmp_data,
               pivot_highs=pivot_highs,
               pivot_lows=pivot_lows,
               pivot_both=pivot_both,
               price_ch=price_ch,
               candle=100,
               backcandles=40,
               window=3
)


In [ ]:
visualize_data(tmp_data,
               pivot_highs=pivot_highs,
               pivot_lows=pivot_lows,
               pivot_both=pivot_both,
               price_ch=price_ch,
               candle=100,
               backcandles=40,
               window=3,
               do_show=False,
)


In [ ]:
visualize_data(tmp_data,
               pivot_highs=pivot_highs,
               pivot_lows=pivot_lows,
               pivot_both=pivot_both,
               price_ch=price_ch,
               candle=100,
               backcandles=40,
               window=3,
               do_show=False,
               save_path="test.svg",
)


### считаем пробития

In [ ]:
tmp_price_ch = collect_channel(df_display, candle, backcandles, window, should_refine=True)
is_breakout_series = calculate_breakout_series(df_display, tmp_price_ch)
# is_breakout_series

In [ ]:
visualize_data(tmp_data,
               pivot_highs=pivot_highs,
               pivot_lows=pivot_lows,
               pivot_both=pivot_both,
               price_ch=tmp_price_ch,
               candle=100,
               backcandles=40,
               window=3,
               show_breakout=True
)


### перебираем границы для ценового канала

In [ ]:
res = analyze_price_channels(tmp_data, len(tmp_data)-1, window, max_backcandles=100, should_refine=False)

In [ ]:
res[2]
len(res)

In [ ]:
from IPython.display import clear_output
from time import sleep


for i in range(len(res)):
    cur_ch = res[-i]["channel"]
    visualize_data(tmp_data,
               pivot_highs=pivot_highs,
               pivot_lows=pivot_lows,
               pivot_both=pivot_both,
               price_ch=cur_ch,
            #    price_ch=refined_channel,
               candle=100,
               backcandles=40,
               window=3,
               show_breakout=True,

    )

    clear_output(wait=True)
    sleep(0.03)


In [ ]:
plt.plot([cur_res['pivot_idx'] for cur_res in res], [cur_res['low_mse'] for cur_res in res])

### Рисуем уровни поддержки и сопротивления

In [ ]:
from help_src.src import calculate_support_levels

support_levels = calculate_support_levels(tmp_data,
                                          static_price_column='Low',
                                          static_window_size=50)
support_levels['static']


In [ ]:
visualize_data(tmp_data,
               pivot_highs=pivot_highs,
               pivot_lows=pivot_lows,
               pivot_both=pivot_both,
               candle=100,
               backcandles=40,
               window=3
)


# Тестируем функции tg_bot

In [ ]:
from help_src.tg_bot.services.advice_service import get_formatted_advice, get_advice_with_price_channel
import asyncio


# Тестируем get_formatted_advice
ticker = "T"
t_token = T_TOKEN

In [ ]:
await get_advice_with_price_channel(ticker, t_token)